# Perfil y código de tiempo

En el proceso de desarrollo de código y de creación de canales de procesamiento de datos, a menudo hay compensaciones que se pueden hacer entre varias implementaciones.
Al principio del desarrollo del algoritmo, puede ser contraproducente preocuparse por estas cosas. Como dijo Donald Knuth: "Deberíamos olvidarnos de las pequeñas eficiencias, digamos el 97% de las veces: la optimización prematura es la raíz de todos los males".

Pero una vez que tienes tu código funcionando, puede ser útil indagar un poco en su eficiencia.
A veces es útil comprobar el tiempo de ejecución de un determinado comando o conjunto de comandos; otras veces es útil profundizar en un proceso multilínea y determinar dónde se encuentra el cuello de botella en alguna complicada serie de operaciones.
IPython proporciona acceso a una amplia gama de funcionalidades para este tipo de cronometraje y perfilado de código.
Aquí discutiremos los siguientes comandos mágicos de IPython:

- ``%time``: Cronometra la ejecución de una única sentencia
- ``%timeit``: Cronometra la ejecución repetida de una única sentencia para mayor precisión
- ``%prun``: Ejecuta el código con el perfilador
- ``%lprun``: Ejecuta el código con el perfilador línea por línea
- ``%memit``: Mide el uso de memoria de una sola sentencia
- ``%mprun``: Ejecuta el código con el perfilador de memoria línea a línea

Los últimos cuatro comandos no están incluidos en IPython - tendrás que conseguir las extensiones ``line_profiler`` y ``memory_profiler``, que discutiremos en las siguientes secciones.

## Fragmentos de Código de Cronometraje: ``%timeit`` y ``%time``

Vimos la magia de línea ``%timeit`` y la magia de celda ``%timeit`` en la introducción a las funciones mágicas en [IPython Magic Commands](01.03-Magic-Commands.ipynb); se puede utilizar para cronometrar la ejecución repetida de fragmentos de código:

In [ ]:
%timeit sum(range(100))

100000 loops, best of 3: 1.54 µs per loop


Ten en cuenta que como esta operación es tan rápida, ``%timeit`` hace automáticamente un gran número de repeticiones.
Para comandos más lentos, ``%timeit`` se ajustará automáticamente y realizará menos repeticiones:

In [ ]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

1 loops, best of 3: 407 ms per loop


A veces, repetir una operación no es la mejor opción.
Por ejemplo, si tenemos una lista que nos gustaría ordenar, podríamos equivocarnos con una operación repetida.
Ordenar una lista preordenada es mucho más rápido que ordenar una lista sin ordenar, por lo que la repetición sesgará el resultado:

In [ ]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

100 loops, best of 3: 1.9 ms per loop


Para esto, la función mágica ``%time`` puede ser una mejor opción. También es una buena opción para comandos de larga duración, cuando es poco probable que los pequeños retrasos relacionados con el sistema afecten al resultado.
Cronometremos la ordenación de una lista sin ordenar y de una lista preordenada:

In [ ]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 40.6 ms, sys: 896 µs, total: 41.5 ms
Wall time: 41.5 ms


In [ ]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 8.18 ms, sys: 10 µs, total: 8.19 ms
Wall time: 8.24 ms


Fíjate en lo rápido que se ordena la lista preclasificada, pero fíjate también en lo mucho que tarda el tiempo con ``%time`` frente a ``%timeit``, ¡incluso para la lista preclasificada!
Esto es el resultado del hecho de que ``%timeit`` hace algunas cosas inteligentes bajo el capó para evitar que las llamadas al sistema interfieran con el tiempo.
Por ejemplo, evita la limpieza de los objetos de Python que no se utilizan (conocida como *recolección de basura*), que de otro modo podría afectar a la sincronización.
Por esta razón, los resultados de ``%timeit`` suelen ser notablemente más rápidos que los de ``%time``.

Para ``%time``, al igual que con ``%timeit``, el uso de la sintaxis mágica de celdas con doble signo permite cronometrar los scripts de varias líneas:

In [ ]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 504 ms, sys: 979 µs, total: 505 ms
Wall time: 505 ms


Para más información sobre ``%time`` y ``%timeit``, así como sus opciones disponibles, utilice la funcionalidad de ayuda de IPython (es decir, escriba ``%time?`` en el prompt de IPython).

## Profiling Full Scripts: ``%prun``

Un programa está hecho de muchas sentencias individuales, y a veces cronometrar estas sentencias en su contexto es más importante que cronometrarlas por separado.
Python contiene un perfilador de código incorporado (que puedes leer en la documentación de Python), pero IPython ofrece una forma mucho más conveniente de usar este perfilador, en la forma de la función mágica ``%prun``.

A modo de ejemplo, definiremos una función simple que hace algunos cálculos:

In [ ]:
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
    return total

Ahora podemos llamar a ``%prun`` con una llamada a la función para ver los resultados perfilados:

In [ ]:
%prun sum_of_lists(1000000)

En el cuaderno, la salida se imprime en el localizador, y tiene el siguiente aspecto

```
14 function calls in 0.714 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.599    0.120    0.599    0.120 <ipython-input-19>:4(<listcomp>)
        5    0.064    0.013    0.064    0.013 {built-in method sum}
        1    0.036    0.036    0.699    0.699 <ipython-input-19>:1(sum_of_lists)
        1    0.014    0.014    0.714    0.714 <string>:1(<module>)
        1    0.000    0.000    0.714    0.714 {built-in method exec}
```

El resultado es una tabla que indica, por orden de tiempo total en cada llamada a la función, dónde está gastando más tiempo la ejecución. En este caso, el grueso del tiempo de ejecución está en la comprensión de la lista dentro de ``sum_of_lists``.
A partir de aquí, podríamos empezar a pensar en qué cambios podríamos hacer para mejorar el rendimiento en el algoritmo.

Para más información sobre ``%prun``, así como sus opciones disponibles, utiliza la funcionalidad de ayuda de IPython (es decir, escribe ``%prun?`` en el prompt de IPython).

## Line-By-Line Profiling with ``%lprun``

The function-by-function profiling of ``%prun`` is useful, but sometimes it's more convenient to have a line-by-line profile report.
This is not built into Python or IPython, but there is a ``line_profiler`` package available for installation that can do this.
Start by using Python's packaging tool, ``pip``, to install the ``line_profiler`` package:

```
$ pip install line_profiler
```

Next, you can use IPython to load the ``line_profiler`` IPython extension, offered as part of this package:

In [ ]:
%load_ext line_profiler

Ahora el comando ``%lprun`` hará un perfilado línea por línea de cualquier función-en este caso, necesitamos decirle explícitamente qué funciones nos interesa perfilar:

In [ ]:
%lprun -f sum_of_lists sum_of_lists(5000)

Al igual que antes, el cuaderno envía el resultado al localizador, pero tiene el siguiente aspecto:

```
Timer unit: 1e-06 s

Total time: 0.009382 s
File: <ipython-input-19-fa2be176cc3e>
Function: sum_of_lists at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def sum_of_lists(N):
     2         1            2      2.0      0.0      total = 0
     3         6            8      1.3      0.1      for i in range(5):
     4         5         9001   1800.2     95.9          L = [j ^ (j >> i) for j in range(N)]
     5         5          371     74.2      4.0          total += sum(L)
     6         1            0      0.0      0.0      return total
```

La información de la parte superior nos da la clave para leer los resultados: el tiempo se reporta en microsegundos y podemos ver dónde el programa pasa más tiempo.
En este punto, podemos utilizar esta información para modificar aspectos del script y hacer que funcione mejor para nuestro caso de uso deseado.

Para obtener más información sobre ``%lprun``, así como sus opciones disponibles, utilice la funcionalidad de ayuda de IPython (es decir, escriba ``%lprun?`` en el prompt de IPython).<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

## Perfilando el uso de memoria: ``%memit`` y ``%mprun``

Otro aspecto del perfilado es la cantidad de memoria que utiliza una operación.
Esto puede ser evaluado con otra extensión de IPython, el ``perfilador de memoria``.
Al igual que con el ``perfilador de líneas``, empezamos instalando la extensión ``pip``:

```
$ pip install memory_profiler
```

Luego podemos usar IPython para cargar la extensión:

In [ ]:
%load_ext memory_profiler

La extensión del perfilador de memoria contiene dos útiles funciones mágicas: la magia ``%memit`` (que ofrece un equivalente de medición de memoria de ``%timeit``) y la función ``%mprun`` (que ofrece un equivalente de medición de memoria de ``%lprun``).
La función ``%memit`` puede utilizarse de forma bastante sencilla:

In [ ]:
%memit sum_of_lists(1000000)

peak memory: 100.08 MiB, increment: 61.36 MiB


We see that this function uses about 100 MB of memory.

For a line-by-line description of memory use, we can use the ``%mprun`` magic.
Unfortunately, this magic works only for functions defined in separate modules rather than the notebook itself, so we'll start by using the ``%%file`` magic to create a simple module called ``mprun_demo.py``, which contains our ``sum_of_lists`` function, with one addition that will make our memory profiling results more clear:

In [ ]:
%%file mprun_demo.py
def sum_of_lists(N):
    total = 0
    for i in range(5):
        L = [j ^ (j >> i) for j in range(N)]
        total += sum(L)
        del L # remove reference to L
    return total

Overwriting mprun_demo.py


Ahora podemos importar la nueva versión de esta función y ejecutar el perfilador de líneas de memoria:

In [ ]:
from mprun_demo import sum_of_lists
%mprun -f sum_of_lists sum_of_lists(1000000)



El resultado, impreso en el localizador, nos da un resumen del uso de la memoria de la función, y se parece a esto:
```
Filename: ./mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     4     71.9 MiB      0.0 MiB           L = [j ^ (j >> i) for j in range(N)]


Filename: ./mprun_demo.py

Line #    Mem usage    Increment   Line Contents
================================================
     1     39.0 MiB      0.0 MiB   def sum_of_lists(N):
     2     39.0 MiB      0.0 MiB       total = 0
     3     46.5 MiB      7.5 MiB       for i in range(5):
     4     71.9 MiB     25.4 MiB           L = [j ^ (j >> i) for j in range(N)]
     5     71.9 MiB      0.0 MiB           total += sum(L)
     6     46.5 MiB    -25.4 MiB           del L # remove reference to L
     7     39.1 MiB     -7.4 MiB       return total
```
Aquí la columna ``Incremento`` nos dice cuánto afecta cada línea al presupuesto total de memoria: observa que cuando creamos y borramos la lista ``L``, estamos añadiendo unos 25 MB de uso de memoria.
Esto se suma al uso de memoria de fondo del propio intérprete de Python.

Para más información sobre ``%memit`` y ``%mprun``, así como sus opciones disponibles, utiliza la funcionalidad de ayuda de IPython (es decir, escribe ``%memit?`` en el prompt de IPython).